Convert TensorFlow.Keras to TensorFlow.Lite

In [15]:
import tensorflow as tf
from tensorflow import keras

In [16]:
model = keras.models.load_model('bees-wasps.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [17]:
tflite_model = converter.convert()

with open('bee-wasp-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Maria\AppData\Local\Temp\tmpe5v1vjdz\assets


INFO:tensorflow:Assets written to: C:\Users\Maria\AppData\Local\Temp\tmpe5v1vjdz\assets


In [18]:
import tensorflow.lite as tflite

In [19]:
interpreter = tflite.Interpreter(model_path='bee-wasp-model.tflite')
interpreter.allocate_tensors()

In [20]:
interpreter.get_input_details()

[{'name': 'serving_default_conv2d_input:0',
  'index': 0,
  'shape': array([  1, 150, 150,   3]),
  'shape_signature': array([ -1, 150, 150,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [21]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 13,
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [22]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [23]:
img = download_image('https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg')
img_150 = prepare_image(img, [150,150])

In [24]:
import numpy as np
x = np.array(img_150, dtype='float32')
X = np.array([x])

In [25]:
def preprocess_input(x):
    x /= 255
    return x

In [26]:
X_prep = preprocess_input(X)
X_prep

array([[[[0.94509804, 0.9098039 , 0.8509804 ],
         [0.9372549 , 0.92156863, 0.9764706 ],
         [0.9137255 , 0.8980392 , 0.95686275],
         ...,
         [0.2901961 , 0.33333334, 0.16470589],
         [0.34901962, 0.40784314, 0.15294118],
         [0.29803923, 0.36078432, 0.11764706]],

        [[0.9490196 , 0.9098039 , 0.8627451 ],
         [0.91764706, 0.9098039 , 0.9607843 ],
         [0.9019608 , 0.89411765, 0.9490196 ],
         ...,
         [0.27450982, 0.3372549 , 0.16078432],
         [0.4627451 , 0.5058824 , 0.19607843],
         [0.4509804 , 0.49411765, 0.18431373]],

        [[0.92941177, 0.88235295, 0.81960785],
         [0.9137255 , 0.9098039 , 0.9647059 ],
         [0.90588236, 0.89411765, 0.9607843 ],
         ...,
         [0.32156864, 0.3764706 , 0.1764706 ],
         [0.49803922, 0.5294118 , 0.20392157],
         [0.43137255, 0.46666667, 0.21176471]],

        ...,

        [[0.03137255, 0.06666667, 0.04705882],
         [0.02352941, 0.08627451, 0.04705882]

In [27]:
pred = model(X_prep)
pred

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.65921414]], dtype=float32)>

In [28]:
interpreter.set_tensor(0, X_prep)
interpreter.invoke()
preds = interpreter.get_tensor(13)

float_predictions = preds[0].tolist()
float_predictions

[0.6592139005661011]

In [10]:
import requests

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'

data = {'url': 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'}

result = requests.post(url, json=data).json()
print(result)

[0.44533514976501465]


My first lambda experience